<a href="https://colab.research.google.com/github/kimjc95/kimjc95.github.io/blob/master/OTUD6B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab

In [1]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [3]:
!mamba install -c conda-forge openmm mdtraj nglview plotly


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['openmm', 'mdtraj', 'nglview', 'plotly']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/n

In [4]:
from openmm import *
from openmm.app import *
from openmm.unit import *

psf = CharmmPsfFile('OTUD6B.psf')
crd = CharmmCrdFile('OTUD6B.crd')

In [5]:
modeller = Modeller(psf.topology, crd.positions)

forcefield = ForceField('charmm36.xml', 'charmm36/water.xml', 'TP2.xml')

modeller.addSolvent(forcefield, model='tip3p', padding=1.2*nanometer, boxShape='dodecahedron', ionicStrength=0.15*molar, neutralize=True)

In [6]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1.2*nanometer, constraints=HBonds, rigidWater=True,
                                 removeCMMotion=True, switchDistance=1.0*nanometer, hydrogenMass=2.0*amu)

In [7]:
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
platform = Platform.getPlatformByName('CUDA')
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)

In [8]:
simulation.minimizeEnergy()

In [9]:
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(modeller.topology, positions, open('minimized.pdb', 'w'))

In [10]:
from tqdm import tqdm

simulation.reporters.append(XTCReporter('nvt_100ps.xtc', 1000))
simulation.reporters.append(StateDataReporter('nvt_log.txt', 1000, step=True, potentialEnergy=True, temperature=True))

for i in tqdm(range(25)):
    simulation.step(1000)

100%|██████████| 25/25 [01:52<00:00,  4.50s/it]


In [11]:
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(modeller.topology, positions, open('nvt_equilibrated.pdb', 'w'))

In [12]:
simulation.reporters.clear()
system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))
simulation.context.reinitialize(preserveState=True)
simulation.context.setStepCount(0)

In [13]:
simulation.reporters.append(XTCReporter('npt_10ns.xtc', 1000))
simulation.reporters.append(StateDataReporter('npt_log.txt', 1000, step=True, potentialEnergy=True, temperature=True, density=True))

for i in tqdm(range(2500)):
    simulation.step(1000)

100%|██████████| 2500/2500 [3:21:48<00:00,  4.84s/it]


In [14]:
import mdtraj as md
import nglview as nv

traj = md.load('npt_10ns.xtc', top='nvt_equilibrated.pdb')

traj.make_molecules_whole(inplace=True)
traj.image_molecules(inplace=True)
traj.remove_solvent(inplace=True)
newTraj = traj.superpose(traj, frame=0)

newTraj[0].save('init.pdb')
newTraj.save_xtc('md.xtc')

In [18]:
from google.colab import output
output.enable_custom_widget_manager()

w = nv.show_mdtraj(newTraj)
w

NGLWidget(max_frame=2499)

In [20]:
import numpy as np

pairs = np.array([[2638, 4524],[4522,4558],[4522,4454],[4522,4455],[4522,4456]])

distances = md.compute_distances(newTraj, pairs)

In [25]:
His_Phos = []

for dis in distances:
    His_Phos.append(min(dis[2:]))

In [28]:
import plotly.graph_objects as go

t = []
SH = []
HO = []

for i, dis in enumerate(distances):
    t.append(4*i)
    SH.append(dis[0])
    HO.append(dis[1])

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=SH, mode='lines', name='Cys158_S - His277_H'))
fig.add_trace(go.Scatter(x=t, y=HO, mode='lines', name='His277_H - Asn279_O'))
fig.add_trace(go.Scatter(x=t, y=His_Phos, mode='lines', name='His277_H - pTyr272_O'))

fig.update_layout(title='Interatomic distances', xaxis_title='picoseconds', yaxis_title='Angstrom')

fig